# Scale-resolved correlation test (Appendix A)

This notebook reproduces Appendix A with explicit density matrices.

We construct two 4-qubit states (ordered as **[L0, L1, R0, R1]**):

- **ρ (aligned):** Bell pairs (L0–R0) and (L1–R1)  
- **σ (cross-wired):** Bell pairs (L0–R1) and (L1–R0)

We verify:

1. Total mutual information is equal:  \(I(L:R)_\rho = I(L:R)_\sigma\)
2. Coarse mutual information differs after coarse-graining (trace out L1,R1): \(I(L0:R0)_\rho \neq I(L0:R0)_\sigma\)
3. The scale profile \(\mathbf{E}=(E_0,E_1)\) differs at fixed total \(I(L:R)\).

No quantum gravity assumptions are used. This is a pure QI check.


In [ ]:
import numpy as np

def ket00(n):
    v = np.zeros((2**n,), dtype=complex)
    v[0] = 1.0
    return v

def bell_state_phi_plus():
    # |Φ+> = (|00> + |11>)/sqrt(2)
    v = np.zeros((4,), dtype=complex)
    v[0] = 1/np.sqrt(2)
    v[3] = 1/np.sqrt(2)
    return v

def dm_from_state(psi):
    psi = psi.reshape((-1,1))
    return psi @ psi.conj().T

def kron(*args):
    out = np.array([[1.0+0j]])
    for a in args:
        out = np.kron(out, a)
    return out

def permute_qubits_density(rho, perm):
    """Permute qubit order for an n-qubit density matrix.

    rho: (2^n,2^n)
    perm: list length n, giving new order in terms of old indices.
          Example perm=[0,2,1,3] means new qubit 0 is old 0, new qubit 1 is old 2, etc.
    """
    n = int(round(np.log2(rho.shape[0])))
    assert rho.shape == (2**n, 2**n)
    assert len(perm) == n

    # Reshape into 2x...x2 (ket) and 2x...x2 (bra)
    rho_t = rho.reshape([2]*n + [2]*n)
    # Axes: [ket0..ket(n-1), bra0..bra(n-1)]
    ket_axes = list(range(n))
    bra_axes = list(range(n, 2*n))

    # We want to reorder ket axes and bra axes in the same way
    new_ket_axes = [ket_axes[i] for i in perm]
    new_bra_axes = [bra_axes[i] for i in perm]
    rho_perm = np.transpose(rho_t, axes=new_ket_axes + new_bra_axes)
    return rho_perm.reshape((2**n, 2**n))

def partial_trace(rho, keep, dims=None):
    """Partial trace keeping subsystems in `keep` (list of indices), tracing out the rest.

    rho: density matrix on n qubits
    keep: indices to keep (0..n-1)
    """
    n = int(round(np.log2(rho.shape[0])))
    if dims is None:
        dims = [2]*n
    keep = list(keep)
    trace = [i for i in range(n) if i not in keep]

    # Reshape into tensor with 2n indices
    reshaped = rho.reshape(dims + dims)
    # Trace out each subsystem i by contracting ket_i with bra_i
    for i in sorted(trace, reverse=True):
        reshaped = np.trace(reshaped, axis1=i, axis2=i+n)
        dims.pop(i)
    d_keep = int(np.prod(dims))
    return reshaped.reshape((d_keep, d_keep))

def von_neumann_entropy(rho, base=2):
    # Ensure Hermitian numerical stability
    rhoH = (rho + rho.conj().T)/2
    evals = np.linalg.eigvalsh(rhoH)
    evals = np.clip(evals.real, 0, 1)
    nz = evals[evals > 1e-12]
    return float(-(nz*np.log(nz)).sum() / np.log(base))

def mutual_information(rho_ab, dims_a, dims_b):
    # rho_ab is joint; dims_a and dims_b are sizes (not qubits)
    # Here dims are powers of 2 in this notebook.
    rho_a = partial_trace(rho_ab, keep=list(range(int(np.log2(dims_a)))), dims=[2]*int(np.log2(dims_a)+np.log2(dims_b)))
    # The above line is awkward; we will instead use a dedicated function for bipartition on qubits below.
    raise NotImplementedError


In [ ]:
# Convenience: mutual information for bipartitions on qubits in the global [L0,L1,R0,R1] register.

def MI_on_qubits(rho, A, B, n=4):
    """Quantum mutual information I(A:B) for subsets A,B of qubit indices."""
    A = sorted(A); B = sorted(B)
    assert set(A).isdisjoint(set(B))
    AB = sorted(A+B)

    rho_AB = partial_trace(rho, keep=AB)
    # Reduced A from AB by tracing out B indices inside AB ordering
    # Map original qubit indices -> positions inside AB
    pos = {q:i for i,q in enumerate(AB)}
    Apos = [pos[q] for q in A]
    Bpos = [pos[q] for q in B]

    rho_A = partial_trace(rho_AB, keep=Apos)
    rho_B = partial_trace(rho_AB, keep=Bpos)

    S_AB = von_neumann_entropy(rho_AB)
    S_A  = von_neumann_entropy(rho_A)
    S_B  = von_neumann_entropy(rho_B)
    return S_A + S_B - S_AB

def log_negativity_two_qubits(rho_2q):
    """Log negativity for a 2-qubit density matrix (base-2)."""
    # Partial transpose on the second qubit
    rho_t = rho_2q.reshape(2,2,2,2)  # a,b,a',b'
    rho_pt = np.transpose(rho_t, axes=(0,3,2,1)).reshape(4,4)  # transpose b and b'
    evals = np.linalg.eigvals(rho_pt)
    # trace norm = sum |eigs|
    tr_norm = float(np.sum(np.abs(evals)))
    return float(np.log2(tr_norm))


In [ ]:
# Build ρ and σ

phi = bell_state_phi_plus()
rho_bell = dm_from_state(phi)

# ρ: Bell on (L0,R0) and (L1,R1)
# First build in ordering [L0,R0,L1,R1], then permute to [L0,L1,R0,R1]
rho_tmp = kron(rho_bell, rho_bell)  # [pair1, pair2] = [L0R0, L1R1]
# Current qubit order is [L0, R0, L1, R1]
rho_aligned = permute_qubits_density(rho_tmp, perm=[0,2,1,3])  # -> [L0, L1, R0, R1]

# σ: Bell on (L0,R1) and (L1,R0)
# Build in ordering [L0,R1,L1,R0], then permute to [L0,L1,R0,R1]
sigma_tmp = kron(rho_bell, rho_bell)  # [L0R1, L1R0] in that order
# Current qubit order is [L0, R1, L1, R0]
sigma_cross = permute_qubits_density(sigma_tmp, perm=[0,2,3,1])  # -> [L0, L1, R0, R1]

# Sanity checks
print('Tr(ρ)=', np.trace(rho_aligned).real)
print('Tr(σ)=', np.trace(sigma_cross).real)
print('ρ Hermitian?', np.allclose(rho_aligned, rho_aligned.conj().T))
print('σ Hermitian?', np.allclose(sigma_cross, sigma_cross.conj().T))


In [ ]:
# Compute total mutual information I(L:R) where L={L0,L1} and R={R0,R1}

L = [0,1]
R = [2,3]

I0_rho = MI_on_qubits(rho_aligned, L, R)
I0_sig = MI_on_qubits(sigma_cross,  L, R)

print('I0 = I(L:R)')
print('  I0(ρ) =', I0_rho)
print('  I0(σ) =', I0_sig)


In [ ]:
# Coarse-grain: keep only coarse qubits L0 and R0 (trace out L1 and R1)
# This corresponds to scale k=1 in Appendix A.

rho_L0R0 = partial_trace(rho_aligned, keep=[0,2])   # keep L0 (0) and R0 (2)
sig_L0R0 = partial_trace(sigma_cross,  keep=[0,2])

I1_rho = MI_on_qubits(rho_aligned, [0], [2])
I1_sig = MI_on_qubits(sigma_cross,  [0], [2])

print('I1 = I(L0:R0)')
print('  I1(ρ) =', I1_rho)
print('  I1(σ) =', I1_sig)

print('\nReduced states:')
print('ρ_L0R0 ≈ Bell?  log-negativity =', log_negativity_two_qubits(rho_L0R0))
print('σ_L0R0 product? log-negativity =', log_negativity_two_qubits(sig_L0R0))


In [ ]:
# Scale profile E = (E0, E1) with I2=0:
# E0 = I0 - I1 ; E1 = I1

E_rho = (I0_rho - I1_rho, I1_rho)
E_sig = (I0_sig - I1_sig, I1_sig)

print('Scale profile E = (E0,E1)')
print('  E(ρ) =', E_rho)
print('  E(σ) =', E_sig)


In [ ]:
# Summary check: equal total MI, different scale profile

print('Equal total MI? ', np.isclose(I0_rho, I0_sig))
print('Different coarse MI? ', not np.isclose(I1_rho, I1_sig))
print('Different scale profiles? ', (not np.allclose(E_rho, E_sig)))


## Interpretation for testing

- **Total mutual information** is identical: both states contain two Bell pairs across the L:R cut.
- Under the **coarse-graining** that keeps only (L0,R0), the aligned state retains one Bell pair at the interface while the cross-wired state retains none.
- The scale profile differs:
  - \(\mathbf{E}(\rho)=(2,2)\)
  - \(\mathbf{E}(\sigma)=(4,0)\)

This is the minimal falsifiable instance of the claim:

> At fixed total correlation, changing *where* correlation lives across scales changes the effective connectivity of constrained protocols.

Next step (if you want it): add noise \(p\), generate curves \(I_0(p), I_1(p), E(p)\), and test robustness.
